# Backend, Frontend

Backend
```
INFO:     Started server process [1660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
Public URL: https://briefly-knowing-treefrog.ngrok-free.app
```

Frontend
```
 VITE v4.5.0  ready in 830 ms


  ➜  Local:   http://localhost:5173/
  ➜  Local:   http://localhost:5173/
  ➜  Network: use --host to expose
  ➜  press h to show help
Browserslist: caniuse-lite is outdated. Please run:
  npx update-browserslist-db@latest
  Why you should do it regularly: https://github.com/browserslist/update-db#readme

🌼 daisyUI components 2.50.0  https://daisyui.com
  ✔︎ Including:  base, components, 3 themes, utilities
  ❤︎ Support daisyUI:  https://opencollective.com/daisyui
```

=======
mà UI run lên ping tới backend ko được???

```
Request URL:
https://toad-vast-civet.ngrok-free.app/rag/nttu?q=sssss
Referrer Policy:
strict-origin-when-cross-origin
```

# 1. Mở CORS - VÌ CỨ TƯỞNG LỖI Ở ĐÂY ==== MẢI ĐI ƠỞ CORS MÀ KO ĐI CHECK CRUL BACKEND TRƯỚC



Code của bạn đã có cấu hình CORS đúng. Tuy nhiên, để xử lý lỗi và làm cho API an toàn hơn, tôi đề xuất một số cải tiến:

```python
from typing import Union, Optional
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from fastapi.encoders import jsonable_encoder
from fastapi import FastAPI, HTTPException

# Định nghĩa các nguồn dữ liệu hợp lệ
VALID_SOURCES = ["nttu", "wiki"]

origins = ["*"]
app_api = FastAPI()
app_api.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app_api.get("/")
def read_root():
    return {"message": "API RAG is running"}

@app_api.get("/rag/{source}")
async def read_item(source: str, q: Optional[str] = None):
    # Kiểm tra source có hợp lệ không
    if source not in VALID_SOURCES:
        raise HTTPException(
            status_code=400, 
            detail=f"Invalid source. Must be one of: {VALID_SOURCES}"
        )
    
    # Kiểm tra q có giá trị không
    if not q:
        raise HTTPException(
            status_code=400,
            detail="Query parameter 'q' is required"
        )
    
    try:
        # Thực hiện truy vấn RAG
        data = app.rag(source=source)(q)
        
        # Xử lý kết quả
        sources = []
        for docs in data["source_documents"]:
            sources.append(docs.to_json()["kwargs"])
        
        res = {
            "result": data["result"],
            "source_documents": sources
        }
        
        return JSONResponse(content=jsonable_encoder(res))
        
    except Exception as e:
        # Xử lý lỗi
        raise HTTPException(
            status_code=500,
            detail=f"An error occurred: {str(e)}"
        )
```

Những cải tiến chính:

1. **Kiểm tra nguồn dữ liệu hợp lệ**:
   - Định nghĩa danh sách `VALID_SOURCES` để kiểm tra source
   - Trả về lỗi 400 nếu source không hợp lệ

2. **Kiểm tra tham số query**:
   - Kiểm tra xem q có giá trị không
   - Trả về lỗi 400 nếu q không được cung cấp

3. **Xử lý lỗi**:
   - Sử dụng try-except để bắt các lỗi có thể xảy ra
   - Trả về lỗi 500 nếu có lỗi trong quá trình xử lý

4. **Response rõ ràng hơn**:
   - Root endpoint trả về message rõ ràng
   - Sử dụng Optional type hint cho tham số q

Những cải tiến này sẽ giúp:
- API trả về thông báo lỗi rõ ràng hơn
- Dễ dàng debug khi có vấn đề
- Bảo vệ API khỏi các input không hợp lệ
- Code dễ bảo trì hơn


CORS (Cross-Origin Resource Sharing) được cấu hình ở **backend**. Nó cho phép hoặc chặn các yêu cầu từ các nguồn khác nhau (cross-origin) đến server của bạn.

### Tại sao cấu hình CORS ở backend?

- **Bảo mật**: Đảm bảo chỉ những nguồn được phép mới có thể truy cập tài nguyên của server.
- **Kiểm soát truy cập**: Bạn có thể chỉ định các domain cụ thể được phép gửi yêu cầu đến server.

### Cách cấu hình CORS:

Trong FastAPI, bạn đã cấu hình CORS như sau:

```python
from fastapi.middleware.cors import CORSMiddleware

app_api.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Cho phép tất cả các nguồn
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
```

### Lưu ý:

- **`allow_origins=["*"]`**: Cho phép tất cả các nguồn. Trong môi trường sản xuất, bạn nên chỉ định các domain cụ thể để tăng cường bảo mật.
- **Frontend**: Không cần cấu hình CORS, nhưng cần đảm bảo rằng các yêu cầu được gửi đến đúng URL của backend.

Nếu bạn có thêm câu hỏi hoặc cần hỗ trợ, hãy cho tôi biết!


# 2. CHECK CRUL BACKEND

## BÁO 404

Để test API backend bằng curl, bạn có thể thử các lệnh sau:

1. **Test endpoint root**:
```bash
curl https://toad-vast-civet.ngrok-free.app/
```

2. **Test endpoint RAG với source "nttu"**:
```bash
curl "https://toad-vast-civet.ngrok-free.app/rag/nttu?q=xin%20chao"
```

3. **Test với headers để tránh lỗi ngrok**:
```bash
curl -H "ngrok-skip-browser-warning: 69420" "https://toad-vast-civet.ngrok-free.app/rag/nttu?q=xin%20chao"
```

4. **Test với source "wiki"**:
```bash
curl -H "ngrok-skip-browser-warning: 69420" "https://toad-vast-civet.ngrok-free.app/rag/wiki?q=xin%20chao"
```

5. **Xem response đầy đủ với headers**:
```bash
curl -v -H "ngrok-skip-browser-warning: 69420" "https://toad-vast-civet.ngrok-free.app/rag/nttu?q=xin%20chao"
```

Chọn một trong các lệnh trên và chạy trong terminal để test. Điều này sẽ giúp bạn xác định xem vấn đề là ở frontend hay backend.


```python
Public URL: https://briefly-knowing-treefrog.ngrok-free.app
INFO:     Started server process [1660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
```

Với thông tin này, bạn nên sử dụng URL ngrok mới để thực hiện các yêu cầu. Hãy thử lại với URL ngrok chính xác:

1. **Test endpoint root**:
```bash
curl https://briefly-knowing-treefrog.ngrok-free.app/
```

2. **Test endpoint RAG với source "nttu"**:
```bash
curl "https://briefly-knowing-treefrog.ngrok-free.app/rag/nttu?q=xin%20chao"
```

3. **Test với headers để tránh lỗi ngrok**:
```bash
curl -H "ngrok-skip-browser-warning: 69420" "https://briefly-knowing-treefrog.ngrok-free.app/rag/nttu?q=xin%20chao"
```

4. **Test với source "wiki"**:
```bash
curl -H "ngrok-skip-browser-warning: 69420" "https://briefly-knowing-treefrog.ngrok-free.app/rag/wiki?q=xin%20chao"
```

5. **Xem response đầy đủ với headers**:
```bash
curl -v -H "ngrok-skip-browser-warning: 69420" "https://briefly-knowing-treefrog.ngrok-free.app/rag/nttu?q=xin%20chao"
```

Hãy thử các lệnh này và xem kết quả. Nếu vẫn gặp lỗi, hãy kiểm tra lại cấu hình endpoint trong code backend.


## Sửa trong frontend

Đúng rồi, bạn cần cập nhật URL trong frontend để khớp với URL ngrok mới. Trong file `ChatBot.jsx`, tìm và sửa đoạn code fetch:

```javascript:front-end/src/components/ChatBot.jsx
// Từ
fetch("https://toad-vast-civet.ngrok-free.app/rag/" + sourceData + "?q=" + promptInput,

// Thành
fetch("https://briefly-knowing-treefrog.ngrok-free.app/rag/" + sourceData + "?q=" + promptInput,
```

Lưu ý:
1. URL ngrok sẽ thay đổi mỗi khi bạn khởi động lại backend, trừ khi bạn sử dụng ngrok với domain tĩnh.
2. Trong môi trường phát triển, bạn có thể muốn tạo một biến môi trường hoặc file cấu hình để dễ dàng thay đổi URL:

```javascript
// Ví dụ tạo biến cho API URL
const API_URL = "https://briefly-knowing-treefrog.ngrok-free.app";

// Sử dụng
fetch(`${API_URL}/rag/${sourceData}?q=${promptInput}`,
```

Sau khi sửa, hãy khởi động lại frontend để áp dụng thay đổi.
